In [2]:
import pandas as pd
import difflib
import json
import warnings
warnings.filterwarnings("ignore")

### Investigação brute force de países em comum

In [8]:
# code that read csv file and save the column "contry name" in a txt file
def read_csv_and_save_column(csv_file, column_name, output_file):
    try:
        df = pd.read_csv(csv_file)
        df = df.sort_values(by=column_name)
        if column_name in df.columns:
            with open(output_file, 'w') as f:
                for item in df[column_name]:
                    f.write(f"{item}\n")
            print(f"Column '{column_name}' saved to '{output_file}'")
        else:
            print(f"Column '{column_name}' not found in the CSV file.")
    except Exception as e:
        print(f"An error occurred: {e}")

for year in range(2019, 2025):
    csv_file = f"./raw_data/world_happiness_{year}.csv"
    output_file = f"./raw_data/world_happiness_{year}_countries.txt"
    read_csv_and_save_column(csv_file, "Country name", output_file)

Column 'Country name' saved to './raw_data/world_happiness_2019_countries.txt'
Column 'Country name' saved to './raw_data/world_happiness_2020_countries.txt'
Column 'Country name' saved to './raw_data/world_happiness_2021_countries.txt'
Column 'Country name' saved to './raw_data/world_happiness_2022_countries.txt'
Column 'Country name' saved to './raw_data/world_happiness_2023_countries.txt'
Column 'Country name' saved to './raw_data/world_happiness_2024_countries.txt'


In [9]:
csv_file = f"./raw_data/regions_flourish.csv"
output_file = f"./raw_data/regions_flourish_countries.txt"
read_csv_and_save_column(csv_file, "Country name", output_file)

Column 'Country name' saved to './raw_data/regions_flourish_countries.txt'


In [27]:
list_delete_2022 = set()
global_list_to_delete = set()

def compare_and_create_global_list(df1, column1, df2, column2, year, list_delete_2022):
    countries1 = df1[column1].values
    countries2 = df2[column2].values

    diff_one_two = set(sorted(countries1)) - set(sorted(countries2))
    diff_two_one = set(sorted(countries2)) - set(sorted(countries1))

    list_delete_2022.update(diff_one_two)
    global_list_to_delete.update(diff_two_one)
    print(f"To delete in 2022 = {diff_one_two}")
    print(f"To delete in {year} = {diff_two_one}")

def compare_with_global_list(df1, column1, df2, column2, year):
    countries1 = df1[column1].values
    countries2 = df2[column2].values

    diff_two_one = set(sorted(countries2)) - set(sorted(countries1))
    noisy_countries = set(global_list_to_delete) - set(diff_two_one)
    print(f"To verify in {year} = {noisy_countries}")

for year in [2019, 2020, 2021, 2023, 2024]:
    print(f"\nSolving differences between 2022 and {year}")
    df1 = pd.read_csv(f"./raw_data/world_happiness_2022.csv")
    df2 = pd.read_csv(f"./raw_data/world_happiness_{year}.csv")
    compare_and_create_global_list(df1, "Country name", df2, "Country name", year, list_delete_2022)

for year in [2019, 2020, 2021, 2023, 2024]:
    print(f"\nSolving name conflicts")
    df1 = pd.read_csv(f"./raw_data/world_happiness_2022.csv")
    df2 = pd.read_csv(f"./raw_data/world_happiness_{year}.csv")
    compare_with_global_list(df1, "Country name", df2, "Country name", year)

print(f"Countries to delete in 2022: {list_delete_2022}")
print(f"Countries to delete according to 2022: {global_list_to_delete}")


Solving differences between 2022 and 2019
To delete in 2022 = {'North Macedonia'}
To delete in 2019 = {'Belarus', 'Swaziland', 'Azerbaijan', 'Macedonia', 'Libya', 'Central African Republic', 'South Sudan', 'Trinidad and Tobago', 'Kuwait', 'Yemen', 'Burundi', 'Haiti', 'Lesotho', 'Rwanda', 'Maldives', 'North Cyprus', 'Turkmenistan'}

Solving differences between 2022 and 2020
To delete in 2022 = {'DR Congo'}
To delete in 2020 = {'Belarus', 'Swaziland', 'Azerbaijan', 'Libya', 'Yemen', 'Kuwait', 'Burundi', 'Haiti', 'Lesotho', 'Rwanda', 'Maldives', 'North Cyprus', 'Turkmenistan'}

Solving differences between 2022 and 2021
To delete in 2022 = {'DR Congo'}
To delete in 2021 = {'Belarus', 'Libya', 'Azerbaijan', 'Yemen', 'Eswatini', 'Kuwait', 'Lesotho', 'Rwanda', 'North Cyprus', 'Turkmenistan'}

Solving differences between 2022 and 2023
To delete in 2022 = set()
To delete in 2023 = {'Libya', 'Azerbaijan', 'Yemen', 'Eswatini', 'Kuwait', 'Lesotho'}

Solving differences between 2022 and 2024
To de

In [ ]:
# substituir em geral = Swaziland: Eswatini, Macedonia: North Macedonia

In [28]:
list_to_sub_general = {"Swaziland": "Eswatini", "Macedonia": "North Macedonia"}
list_to_delete_general = ['DR Congo', 'Azerbaijan', 'Libya', 'Central African Republic', 'South Sudan', 'Eswatini', 'Haiti', 'Rwanda', 'Belize', 'North Cyprus', 'Turkmenistan', 'Belarus', 'Somalia', 'Macedonia', 'Yemen', 'Trinidad and Tobago', 'Kuwait', 'Burundi', 'Lesotho', 'Maldives', 'Oman']

In [29]:
for year in [2019, 2020, 2021, 2022, 2023, 2024]:
    df_world_happiness = pd.read_csv(f"./raw_data/world_happiness_{year}.csv")
    df_world_happiness["Country name"].replace(list_to_sub_general, inplace=True)

    list_new_countries_happiness = df_world_happiness["Country name"].values

    df_world_happiness = df_world_happiness[~df_world_happiness["Country name"].isin(list_to_delete_general)]
    print(f"Final total number of countries in {year}: {len(df_world_happiness)}")

    print(f"Saving {year} record to file")
    df_world_happiness.to_csv(f"./norm_data_2/world_happiness_{year}.csv", index=False)

Final total number of countries in 2019: 136
Saving 2019 record to file
Final total number of countries in 2020: 136
Saving 2020 record to file
Final total number of countries in 2021: 136
Saving 2021 record to file
Final total number of countries in 2022: 136
Saving 2022 record to file
Final total number of countries in 2023: 136
Saving 2023 record to file
Final total number of countries in 2024: 136
Saving 2024 record to file


In [30]:
df_2019 = pd.read_csv("./norm_data_2/world_happiness_2019.csv")
df_2024 = pd.read_csv("./norm_data_2/world_happiness_2024.csv")
df_2019 = df_2019.sort_values(by="Country name")
df_2024 = df_2024.sort_values(by="Country name")
print(df_2019["Country name"].values)
print(df_2024["Country name"].values)

['Afghanistan' 'Albania' 'Algeria' 'Argentina' 'Armenia' 'Australia'
 'Austria' 'Bahrain' 'Bangladesh' 'Belgium' 'Benin' 'Bolivia'
 'Bosnia and Herzegovina' 'Botswana' 'Brazil' 'Bulgaria' 'Burkina Faso'
 'Cambodia' 'Cameroon' 'Canada' 'Chad' 'Chile' 'China' 'Colombia'
 'Comoros' 'Congo' 'Costa Rica' 'Croatia' 'Cyprus' 'Czechia'
 'Côte d’Ivoire' 'Denmark' 'Dominican Republic' 'Ecuador' 'Egypt'
 'El Salvador' 'Estonia' 'Ethiopia' 'Finland' 'France' 'Gabon' 'Gambia'
 'Georgia' 'Germany' 'Ghana' 'Greece' 'Guatemala' 'Guinea' 'Honduras'
 'Hong Kong SAR of China' 'Hungary' 'Iceland' 'India' 'Indonesia' 'Iran'
 'Iraq' 'Ireland' 'Israel' 'Italy' 'Jamaica' 'Japan' 'Jordan' 'Kazakhstan'
 'Kenya' 'Kosovo' 'Kyrgyzstan' 'Lao PDR' 'Latvia' 'Lebanon' 'Liberia'
 'Lithuania' 'Luxembourg' 'Madagascar' 'Malawi' 'Malaysia' 'Mali' 'Malta'
 'Mauritania' 'Mauritius' 'Mexico' 'Mongolia' 'Montenegro' 'Morocco'
 'Mozambique' 'Myanmar' 'Namibia' 'Nepal' 'Netherlands' 'New Zealand'
 'Nicaragua' 'Niger' 'Nigeria' 

In [ ]:
# Normalização dos nomes com o flourish



### Investigação de países com nomes diferentes ou que não existem, entre 2019 e os dados do Flourish

In [18]:
df_happiness = pd.read_csv("./raw_data/world_happiness_2019.csv")
df_regions_flourish = pd.read_csv("./raw_data/regions_flourish.csv")

list_countries_happiness = df_happiness["Country name"].values
list_countries_flourish = df_regions_flourish["Country name"].values

list_in_happiness = list(set(list_countries_happiness) - set(list_countries_flourish)) 

dict_country_name_errors_2019 = {}
for country in list_in_happiness:
    similar_country = difflib.get_close_matches(country, list_countries_flourish, n=20, cutoff=0)
    
    caught_similar_country = False
    for sim_country in similar_country:
        if sim_country.lower() in country.lower() or country.lower() in sim_country.lower():
            dict_country_name_errors_2019[country] = sim_country
            caught_similar_country = True
            break

    if not caught_similar_country:
        dict_country_name_errors_2019[country] = similar_country[:10]

for key in dict_country_name_errors_2019.keys():
    print(f"{key} = {dict_country_name_errors_2019[key]}")

Congo = Republic of the Congo
Republic of Moldova = Moldova
North Cyprus = Cyprus
State of Palestine = Palestine
DR Congo = ['Togo', 'Mongolia', 'Republic of the Congo', 'Hong Kong', 'Tonga', 'Montenegro', 'Monaco', 'Angola', 'Romania', 'Democratic Republic of the Congo']
Hong Kong SAR of China = Hong Kong
Gambia = The Gambia
Republic of Korea = ['Republic of the Congo', 'Democratic Republic of the Congo', 'Czech Republic', 'South Korea', 'North Korea', 'Dominican Republic', 'Isle of Man', 'Central African Republic', 'Papua New Guinea', 'Russia']
Taiwan Province of China = China
Russian Federation = Russia
United States = United States of America
Türkiye = ['Turkey', 'Ukraine', 'Tokelau', 'Eritrea', 'Turkmenistan', 'Suriname', 'Niue', 'Tajikistan', 'Niger', 'Libya']
Lao PDR = ['Laos', 'Lebanon', 'Costa Rica', 'Samoa', 'Puerto Rico', 'Libya', 'Gabon', 'Kingman Reef', 'Latvia', 'Faroe Islands']
Côte d’Ivoire = ['Bonaire', 'United Arab Emirates', 'East Timor', 'Cape Verde', 'Ivory Coast',

In [19]:
dict_sub_country_from_happiness_2019 = {
    'Swaziland': 'Eswatini',
    'DR Congo': 'Democratic Republic of the Congo',
    'Czechia': 'Czech Republic',
    'Congo': 'Republic of the Congo',
    'State of Palestine': 'Palestine',
    'Viet Nam': 'Vietnam',
    'Russian Federation': 'Russia',
    'Lao PDR': 'Laos',
    'Gambia': 'The Gambia',
    'Türkiye': 'Turkey',
    'Republic of Moldova': 'Moldova',
    'Republic of Korea': 'South Korea',
    'United States': 'United States of America',
    'Côte d’Ivoire': 'Ivory Coast',
    'Macedonia': 'North Macedonia',
    'Hong Kong SAR of China': 'Hong Kong'
}

In [20]:
df_happiness_cp = df_happiness.copy()
df_happiness_cp["Country name"].replace(dict_sub_country_from_happiness_2019, inplace=True)

list_new_countries_happiness = df_happiness_cp["Country name"].values
list_to_delete_countries_2019 = list(set(list_new_countries_happiness) - set(list_countries_flourish))
print(f"Countries to delete from 2019 data: {list_to_delete_countries_2019}")

df_happiness_cp_cleaned = df_happiness_cp[~df_happiness_cp["Country name"].isin(list_to_delete_countries_2019)]
print(f"Final total number of countries in 2019: {len(df_happiness_cp_cleaned)}")

Countries to delete from 2019 data: ['Taiwan Province of China', 'North Cyprus']
Final total number of countries in 2019: 151


In [21]:
with open("./raw_data/common_countries_2019_flourish.txt", "w") as f:
    for country in df_happiness_cp_cleaned["Country name"].values:
        f.write(country + "\n")

### Processamento dos conjuntos de acordo com a relação encontrada entre 2019 e dados do Flourish

In [22]:
def normalize_countries(df_happiness: pd.DataFrame) -> pd.DataFrame:
    try:
        df_happiness_cp = df_happiness.copy()
        df_happiness_cp_cleaned = df_happiness_cp[~df_happiness_cp["Country name"].isin(list_to_delete_countries_2019)]
        df_happiness_cp_cleaned["Country name"].replace(dict_sub_country_from_happiness_2019, inplace=True)
        
        return df_happiness_cp_cleaned
    except Exception as err:
        print(f"Erro: {err}")

def check_result(df_happiness_cp_cleaned: pd.DataFrame):
    with open("./raw_data/common_countries_2019_flourish.txt", 'r') as file:
        lines = file.readlines()
    list_choosen_contries = [line.strip() for line in lines]
    
    common_countries_with_2019 = df_happiness_cp_cleaned["Country name"].isin(list_choosen_contries)
    print(f"# common countries: {len(df_happiness_cp_cleaned[common_countries_with_2019])}")

    countries_not_found = ~df_happiness_cp_cleaned["Country name"].isin(list_choosen_contries)
    len_not_found = len(df_happiness_cp_cleaned[countries_not_found])
    print(f"# not found contries: {len_not_found}")

    if len_not_found > 0:
        print(f"-> {df_happiness_cp_cleaned[countries_not_found]['Country name'].tolist()}")

def preprocess_dataframe(year: int) -> pd.DataFrame:
    df_happiness = pd.read_csv(f"./raw_data/world_happiness_{year}.csv")
    df_happiness_cp_cleaned = normalize_countries(df_happiness=df_happiness)
    return df_happiness_cp_cleaned

In [23]:
list_years = [2019, 2020, 2021, 2022, 2023, 2024]

for year in list_years:
    print(f"Processing data from {year}")
    df_happiness_cp_cleaned = preprocess_dataframe(year=year)
    check_result(df_happiness_cp_cleaned=df_happiness_cp_cleaned)
    
    print("Saving dataframe\n")
    df_happiness_cp_cleaned.to_csv(f'./norm_data/world_happiness_{year}.csv', index=False)

Processing data from 2019
# common countries: 151
# not found contries: 0
Saving dataframe

Processing data from 2020
# common countries: 147
# not found contries: 0
Saving dataframe

Processing data from 2021
# common countries: 144
# not found contries: 0
Saving dataframe

Processing data from 2022
# common countries: 136
# not found contries: 0
Saving dataframe

Processing data from 2023
# common countries: 142
# not found contries: 0
Saving dataframe

Processing data from 2024
# common countries: 143
# not found contries: 3
-> ['Belize', 'Oman', 'Somalia']
Saving dataframe



In [24]:
# Remoção dos paises não encontrados em 2024

df_happiness_2024 = pd.read_csv("./norm_data/world_happiness_2024.csv")
removal_not_found = ~df_happiness_2024["Country name"].isin(['Belize', 'Oman', 'Somalia'])
df_happiness_2024 = df_happiness_2024[removal_not_found]
df_happiness_2024.to_csv('./norm_data/world_happiness_2024_v2.csv', index=False)
len(df_happiness_2024)

143

In [25]:
# Ajuste para 2022 com menor número de países em comum: 136

df_happiness_2022 = pd.read_csv("./norm_data/world_happiness_2022.csv")
list_countries_2022 = df_happiness_2022["Country name"].values
print(len(list_countries_2022))

136


In [26]:
for year in [2019, 2020, 2021, 2023, 2024]:
    if year == 2024:
        df_happiness_year = pd.read_csv(f"./norm_data/world_happiness_{year}_v2.csv")
    else:
        df_happiness_year = pd.read_csv(f"./norm_data/world_happiness_{year}.csv")

    filtered_countries = df_happiness_year["Country name"].isin(list_countries_2022)
    df_happiness_year = df_happiness_year[filtered_countries]
    df_happiness_year.to_csv(f'./final_data/world_happiness_{year}.csv', index=False)

In [27]:
# Verificação dos países em comum

df_happiness_2019 = pd.read_csv("./final_data/world_happiness_2019.csv")
df_happiness_2020 = pd.read_csv("./final_data/world_happiness_2020.csv")
df_happiness_2021 = pd.read_csv("./final_data/world_happiness_2021.csv")
df_happiness_2022 = pd.read_csv("./final_data/world_happiness_2022.csv")
df_happiness_2023 = pd.read_csv("./final_data/world_happiness_2023.csv")
df_happiness_2024 = pd.read_csv("./final_data/world_happiness_2024.csv")

In [28]:
set_2019 = set(df_happiness_2019["Country name"].values)
set_2020 = set(df_happiness_2020["Country name"].values)
set_2021 = set(df_happiness_2021["Country name"].values)
set_2022 = set(df_happiness_2022["Country name"].values)
set_2023 = set(df_happiness_2023["Country name"].values)
set_2024 = set(df_happiness_2024["Country name"].values)

In [29]:
def remove_dr_congo(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df = df[df["Country name"] != "Democratic Republic of the Congo"]
    return df

df_happiness_2019 = remove_dr_congo(df_happiness_2019)
df_happiness_2020 = remove_dr_congo(df_happiness_2020)
df_happiness_2021 = remove_dr_congo(df_happiness_2021)
df_happiness_2022 = remove_dr_congo(df_happiness_2022)
df_happiness_2023 = remove_dr_congo(df_happiness_2023)
df_happiness_2024 = remove_dr_congo(df_happiness_2024)

In [30]:
df_happiness_2019.to_csv("./final_data/world_happiness_2019.csv", index=False)
df_happiness_2020.to_csv("./final_data/world_happiness_2020.csv", index=False)
df_happiness_2021.to_csv("./final_data/world_happiness_2021.csv", index=False)
df_happiness_2022.to_csv("./final_data/world_happiness_2022.csv", index=False)
df_happiness_2023.to_csv("./final_data/world_happiness_2023.csv", index=False)
df_happiness_2024.to_csv("./final_data/world_happiness_2024.csv", index=False)

In [3]:
df_happiness_2019 = pd.read_csv("./final_data/world_happiness_2019.csv")
df_happiness_2020 = pd.read_csv("./final_data/world_happiness_2020.csv")
df_happiness_2021 = pd.read_csv("./final_data/world_happiness_2021.csv")
df_happiness_2022 = pd.read_csv("./final_data/world_happiness_2022.csv")
df_happiness_2023 = pd.read_csv("./final_data/world_happiness_2023.csv")
df_happiness_2024 = pd.read_csv("./final_data/world_happiness_2024.csv")

In [7]:
df_cloropletic_map = pd.DataFrame()

df_cloropletic_map["Country name"] = df_happiness_2019["Country name"].to_numpy()
df_cloropletic_map["2019"] = df_happiness_2019["Explained by: Freedom to make life choices"].to_numpy()
df_cloropletic_map["2020"] = df_happiness_2020["Explained by: Freedom to make life choices"].to_numpy()
df_cloropletic_map["2021"] = df_happiness_2021["Explained by: Freedom to make life choices"].to_numpy()
df_cloropletic_map["2022"] = df_happiness_2022["Explained by: Freedom to make life choices"].to_numpy()
df_cloropletic_map["2023"] = df_happiness_2023["Explained by: Freedom to make life choices"].to_numpy()
df_cloropletic_map["2024"] = df_happiness_2024["Explained by: Freedom to make life choices"].to_numpy()
df_cloropletic_map

,Country name,2019,2020,2021,2022,2023,2024
0,Afghanistan,"0,000","0,000","0,000","0,000","0,000","0,986"
1,Albania,"0,462","0,491","0,511","0,549","0,690","0,955"
2,Algeria,"0,084","0,119","0,146","0,252","0,247","0,971"
3,Argentina,"0,521","0,544","0,555","0,587","0,681","0,952"
4,Armenia,"0,378","0,540","0,558","0,551","0,650","0,860"
...,...,...,...,...,...,...,...
130,Uzbekistan,"0,693","0,716","0,716","0,740","0,835","0,598"
131,Venezuela,"0,272","0,284","0,283","0,369","0,518","0,677"
132,Vietnam,"0,651","0,679","0,707","0,741","0,843","0,279"
133,Zambia,"0,491","0,487","0,525","0,545","0,727","0,613"


In [8]:
# convert string columns to numeric
for col in df_cloropletic_map.columns[1:]:
    df_cloropletic_map[col] = df_cloropletic_map[col].str.replace(',', '.').astype(float).fillna(0.0)
df_cloropletic_map

,Country name,2019,2020,2021,2022,2023,2024
0,Afghanistan,0.000,0.000,0.000,0.000,0.000,0.986
1,Albania,0.462,0.491,0.511,0.549,0.690,0.955
2,Algeria,0.084,0.119,0.146,0.252,0.247,0.971
3,Argentina,0.521,0.544,0.555,0.587,0.681,0.952
4,Armenia,0.378,0.540,0.558,0.551,0.650,0.860
...,...,...,...,...,...,...,...
130,Uzbekistan,0.693,0.716,0.716,0.740,0.835,0.598
131,Venezuela,0.272,0.284,0.283,0.369,0.518,0.677
132,Vietnam,0.651,0.679,0.707,0.741,0.843,0.279
133,Zambia,0.491,0.487,0.525,0.545,0.727,0.613


In [9]:
df_cloropletic_map.to_csv("./final_data/world_happiness_freedom.csv", index=False)